# stSME clustering tutorial

**stSME** is a novel normalisation method implemented in *stLearn* software.   
It's designed for **s**patial **t**ranscriptomics data and utilised tissue **S**patial location, **M**orphology, , and gene **E**xpression.

This tutorial demonstrates how to use *stLearn* to perform **stSME** clustering for spatial transcriptomics data

In this tutorial we first focus on **Mouse Brain (Coronal)** Visium dataset from [10x genomics website](https://support.10xgenomics.com/spatial-gene-expression/datasets/1.0.0/V1_Adult_Mouse_Brain).

## Mouse Brain (Coronal)

### 1. Preparation

In [ ]:
import scanpy as sc
import stlearn as st

st.settings.set_figure_params(dpi=180)

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
mouse_brain_coronal = sc.datasets.visium_sge(sample_id="V1_Adult_Mouse_Brain")
mouse_brain_coronal = st.convert_scanpy(mouse_brain_coronal)

In [ ]:
# pre-processing for gene count table
st.pp.filter_genes(mouse_brain_coronal, min_cells=1)
st.pp.normalize_total(mouse_brain_coronal)
st.pp.log1p(mouse_brain_coronal)

In [ ]:
# pre-processing for spot image
st.pp.tiling(mouse_brain_coronal, out_path="tiling")

# this step uses deep learning model to extract high-level features from tile images
# may need few minutes to be completed
st.pp.extract_feature(mouse_brain_coronal)

### 2. run stSME clustering

In [ ]:
# run PCA for gene expression data
st.em.run_pca(mouse_brain_coronal, n_comps=50)

In [ ]:
mouse_brain_coronal_sme = mouse_brain_coronal.copy()
# apply stSME to normalise log transformed data
st.spatial.SME.SME_normalize(mouse_brain_coronal_sme, use_data="raw")
mouse_brain_coronal_sme.X = mouse_brain_coronal_sme.obsm['raw_SME_normalized']
st.pp.scale(mouse_brain_coronal_sme)
st.em.run_pca(mouse_brain_coronal_sme, n_comps=50)

In [ ]:
# K-means clustering on stSME normalised PCA
st.tl.clustering.kmeans(mouse_brain_coronal_sme, n_clusters=19, use_data="X_pca", key_added="X_pca_kmeans")
st.pl.cluster_plot(mouse_brain_coronal_sme, use_label="X_pca_kmeans")

In [ ]:
# louvain clustering on stSME normalised data
st.pp.neighbors(mouse_brain_coronal_sme, n_neighbors=17, use_rep='X_pca')
st.tl.clustering.louvain(mouse_brain_coronal_sme, resolution=1.19)
st.pl.cluster_plot(mouse_brain_coronal_sme, use_label="louvain")

we now move to Mouse Brain (Sagittal Posterior) Visium dataset from [10x genomics website](https://support.10xgenomics.com/spatial-gene-expression/datasets/1.0.0/V1_Mouse_Brain_Sagittal_Posterior).

## Mouse Brain (Sagittal Posterior)

### 1. Preparation

In [ ]:
mouse_brain_sagittal = sc.datasets.visium_sge(sample_id="V1_Mouse_Brain_Sagittal_Posterior")
mouse_brain_sagittal = st.convert_scanpy(mouse_brain_sagittal)

In [ ]:
# pre-processing for gene count table
st.pp.filter_genes(mouse_brain_sagittal, min_cells=1)
st.pp.normalize_total(mouse_brain_sagittal)
st.pp.log1p(mouse_brain_sagittal)
st.pp.scale(mouse_brain_sagittal)

In [ ]:
# pre-processing for spot image
st.pp.tiling(mouse_brain_sagittal, out_path="tiling")

# this step uses deep learning model to extract high-level features from tile images
# may need few minutes to be completed
st.pp.extract_feature(mouse_brain_sagittal)

### 2. run stSME clustering

In [ ]:
# run PCA for gene expression data
st.em.run_pca(mouse_brain_sagittal, n_comps=50)

In [ ]:
mouse_brain_sagittal_sme = mouse_brain_sagittal.copy()
# apply stSME to normalise log transformed data
# with weights from morphological Similarly and physcial distance
st.spatial.SME.SME_normalize(mouse_brain_sagittal_sme, use_data="raw",
                             weights="weights_matrix_pd_md")
mouse_brain_sagittal_sme.X = mouse_brain_sagittal_sme.obsm['raw_SME_normalized']
st.pp.scale(mouse_brain_sagittal_sme)
st.em.run_pca(mouse_brain_sagittal_sme, n_comps=50)

In [ ]:
# K-means clustering on stSME normalised PCA
st.tl.clustering.kmeans(mouse_brain_sagittal_sme, n_clusters=17, use_data="X_pca", key_added="X_pca_kmeans")
st.pl.cluster_plot(mouse_brain_sagittal_sme, use_label="X_pca_kmeans")

In [ ]:
# louvain clustering on stSME normalised data
st.pp.neighbors(mouse_brain_sagittal_sme, n_neighbors=20, use_rep='X_pca')
st.tl.clustering.louvain(mouse_brain_sagittal_sme)
st.pl.cluster_plot(mouse_brain_sagittal_sme, use_label="louvain")

#### Tutorial by Xiao Tan